# 02 - Pipeline Silver (Transformação e Modelagem)
## Camada Silver: Cleaned & Normalized Data

### O que é a Camada Silver?
Se a camada Bronze é um "armazém de dados brutos", a camada Silver é a **"fábrica de processamento"**. Aqui, pegamos aquelas "cartas lacradas" (JSONs) que salvamos na Bronze e:
- **Abrimos** e extraímos as informações importantes
- **Limpamos** dados inconsistentes ou inválidos
- **Organizamos** em tabelas estruturadas (como planilhas do Excel bem organizadas)
- **Validamos** a qualidade dos dados

### Por que precisamos dela?
- **Dados prontos para uso**: Transformamos JSON bagunçado em tabelas limpas
- **Modelagem dimensional**: Criamos estruturas otimizadas para análises (Star Schema)
- **Qualidade garantida**: Aplicamos regras de validação e limpeza
- **Performance**: Dados normalizados são muito mais rápidos de consultar

### O que este notebook faz?
1. **Lê os dados brutos** salvos na camada Bronze
2. **"Abre" os JSONs** (parse) e extrai campos específicos
3. **Cria 3 dimensões**: Países, Moedas, Idiomas
4. **Cria 1 tabela fato**: Métricas dos países (população, área, densidade)
5. **Valida a qualidade**: Verifica integridade, ranges válidos, duplicatas
6. **Otimiza para performance**: Compacta e organiza os dados

### Modelagem: Star Schema
```
       dim_currencies     dim_languages
              \              /
               \            /
                \          /
           fact_country_metrics  ← Tabela Central (Fato)
                     |
                     |
               dim_countries
```


## 1. Preparação do Ambiente

### O que vai acontecer aqui?
Assim como no notebook Bronze, precisamos preparar nosso ambiente com todas as ferramentas e configurações necessárias.

### O que vamos preparar?
1. **Bibliotecas**: Mesmas ferramentas do Bronze (PySpark, datetime, json)
2. **Nomes das tabelas**: Onde vamos LER (Bronze) e ESCREVER (Silver)
3. **Funções auxiliares**: Reutilizando as mesmas "receitas" de código

### Diferença do Bronze:
Agora temos **4 tabelas na camada Silver**:
- `dim_countries`: Informações dos países (dimensão)
- `dim_currencies`: Moedas do mundo (dimensão)
- `dim_languages`: Idiomas do mundo (dimensão)
- `fact_country_metrics`: Métricas populacionais (fato)


In [0]:
# Importações
import requests
import json
from datetime import datetime, date
from pyspark.sql.functions import *
from pyspark.sql.types import *
import time

# Configurações (inline)
SCHEMAS = {
    'bronze': 'workspace.bronze',
    'silver': 'workspace.silver',
    'gold': 'workspace.gold'
}

BRONZE_TABLES = {
    'countries': f"{SCHEMAS['bronze']}.countries_raw",
    'exchange_rates': f"{SCHEMAS['bronze']}.exchange_rates_raw"
}

SILVER_TABLES = {
    'dim_countries': f"{SCHEMAS['silver']}.dim_countries",
    'dim_currencies': f"{SCHEMAS['silver']}.dim_currencies",
    'dim_languages': f"{SCHEMAS['silver']}.dim_languages",
    'fact_country_metrics': f"{SCHEMAS['silver']}.fact_country_metrics"
}

# Funções utilitárias
def log_metrics(stage, table_name, record_count, execution_time=None):
    """Loga métricas de execução"""
    print(f"\n{'='*60}")
    print(f"MÉTRICAS - {stage.upper()}")
    print(f"{'='*60}")
    print(f"Tabela: {table_name}")
    print(f"Registros: {record_count:,}")
    if execution_time:
        print(f"Tempo de execução: {execution_time:.2f} segundos")
    print(f"Timestamp: {datetime.now()}")
    print(f"{'='*60}\n")

def create_database_if_not_exists(spark, schema_name):
    """Cria database se não existir"""
    spark.sql(f"CREATE DATABASE IF NOT EXISTS {schema_name}")
    print(f"Database '{schema_name}' verificado/criado com sucesso")

def optimize_table(spark, table_name, zorder_columns=None):
    """Otimiza tabela Delta com OPTIMIZE e Z-ORDER"""
    print(f"Otimizando tabela {table_name}...")
    if zorder_columns:
        zorder_cols = ', '.join(zorder_columns)
        spark.sql(f"OPTIMIZE {table_name} ZORDER BY ({zorder_cols})")
    else:
        spark.sql(f"OPTIMIZE {table_name}")
    print(f"Tabela {table_name} otimizada com sucesso")

print("✓ Configurações e funções carregadas com sucesso!")

✓ Configurações e funções carregadas com sucesso!


### Iniciando o Processo Silver

**O que este código faz?**
- Marca o momento exato de início do pipeline Silver
- Registra quando começamos o processamento e transformação dos dados

**Por que é importante?**
Assim saberemos quanto tempo leva para transformar 195 países brutos em tabelas limpas e organizadas.


In [0]:
import time
start_time = time.time()

print("Pipeline Silver - Iniciado")
print(f"Timestamp: {datetime.now()}")
print(f"Data de Execução: {date.today()}")

Pipeline Silver - Iniciado
Timestamp: 2025-12-06 14:54:44.285414
Data de Execução: 2025-12-06


## 2. Criando o "Armazém" Silver

### O que estamos fazendo?
Criando a pasta `workspace.silver` onde vamos guardar nossas 4 tabelas limpas e organizadas.

### Por que uma pasta separada?
Separar Bronze e Silver facilita:
- **Organização**: Dados brutos em uma pasta, dados limpos em outra
- **Segurança**: Se algo der errado na Silver, a Bronze permanece intacta
- **Governança**: Controle de acesso diferente por camada


In [0]:
# Criar schema silver se não existir
create_database_if_not_exists(spark, SCHEMAS['silver'])

Database 'workspace.silver' verificado/criado com sucesso


## 3. Lendo os Dados da Camada Bronze

### O que estamos fazendo?
Vamos buscar aquelas "cartas lacradas" (JSONs) que salvamos na Bronze para começar a processá-las.

### Passo a passo:
1. **Lemos a tabela Bronze** completa
2. **Identificamos a data mais recente**: Se o pipeline rodou várias vezes, queremos apenas os dados mais atuais
3. **Filtramos apenas a última execução**: Evitamos processar dados antigos desnecessariamente
4. **Contamos quantos registros** temos para processar

### Por que filtrar pela data mais recente?
Imagine que rodamos o pipeline ontem (195 países) e hoje (195 países). Teríamos 390 registros na Bronze. Mas queremos processar apenas os dados mais atuais (hoje = 195). Isso evita duplicatas e garante que estamos trabalhando com a informação mais fresca.

### Analogia:
É como ir ao correio buscar suas cartas. Se você tem cartas de ontem e de hoje, você pega apenas as de hoje para processar.


In [0]:
print("\n" + "="*60)
print("LEITURA: CAMADA BRONZE")
print("="*60)

# Ler dados mais recentes de countries
df_bronze_countries = spark.table(BRONZE_TABLES['countries'])

# Pegar apenas a execução mais recente e deduplic

latest_date = df_bronze_countries.agg(max('execution_date')).collect()[0][0]
df_bronze_latest = df_bronze_countries.filter(col('execution_date') == latest_date) \
    .dropDuplicates(['data'])  # Garantir 1 linha por pais (caso multiplas execucoes no mesmo dia)

bronze_count = df_bronze_latest.count()
print(f"✓ Registros lidos (data: {latest_date}): {bronze_count}")


LEITURA: CAMADA BRONZE
✓ Registros lidos (data: 2025-12-06): 195


## 4. "Abrindo as Cartas" - Parse do JSON

### O que é "parse"?
**Parse** significa "abrir e interpretar" o JSON. Lembra que na Bronze guardamos tudo como texto (string)? Agora vamos "abrir" esse texto e extrair as informações individuais.

### Analogia completa:
Imagine que cada país é uma carta lacrada com um formulário complexo dentro:

**Antes (Bronze):**
```
Carta lacrada: "{name:{common:'Brazil',official:'República Federativa do Brasil'}, population:213993437, ...}"
```

**Depois (Silver):**
```
Nome comum: Brazil
Nome oficial: República Federativa do Brasil  
População: 213.993.437
...
```

### O que este código faz?

1. **Define a estrutura esperada do JSON** (schema): É como ter um molde que diz "aqui terá um nome, aqui uma população, aqui coordenadas, etc."

2. **"Abre" o JSON** usando esse molde: Transforma o texto em estrutura de dados que o computador entende

3. **Permite acessar campos individuais**: Agora podemos pegar só o nome, só a população, só a capital, etc.

### Por que precisamos de um schema?
O JSON de cada país tem campos aninhados (dentro de outros campos). Por exemplo:
- `name.common` (nome está dentro de outro campo)
- `currencies.BRL.name` (moedas é um dicionário com outros dicionários dentro)

O schema diz ao PySpark: "olha, aqui dentro tem mais coisas, e é assim que você acessa".

### Campos que estamos extraindo:
- **name**: Nome comum e oficial do país
- **cca2/cca3**: Códigos de 2 e 3 letras (BR, BRA)
- **capital**: Cidade capital
- **region/subregion**: Continente e sub-região
- **population**: Número de habitantes
- **area**: Área em km²
- **landlocked**: Se não tem costa marítima
- **latlng**: Coordenadas (latitude, longitude)
- **currencies**: Moedas utilizadas
- **languages**: Idiomas falados


In [0]:
print("\n" + "="*60)
print("TRANSFORMAÇÃO: PARSE JSON")
print("="*60)

# Parse do JSON
df_parsed = df_bronze_latest.withColumn('json_data', from_json(col('data'), 'map<string,string>'))

# Schema completo para parse detalhado
country_schema = StructType([
    StructField('name', StructType([
        StructField('common', StringType()),
        StructField('official', StringType())
    ])),
    StructField('cca2', StringType()),
    StructField('cca3', StringType()),
    StructField('ccn3', StringType()),
    StructField('capital', ArrayType(StringType())),
    StructField('region', StringType()),
    StructField('subregion', StringType()),
    StructField('population', LongType()),
    StructField('area', DoubleType()),
    StructField('landlocked', BooleanType()),
    StructField('latlng', ArrayType(DoubleType())),
    StructField('timezones', ArrayType(StringType())),
    StructField('currencies', MapType(StringType(), StructType([
        StructField('name', StringType()),
        StructField('symbol', StringType())
    ]))),
    StructField('languages', MapType(StringType(), StringType()))
])

# Parse com schema estruturado
df_countries = df_bronze_latest.withColumn(
    'parsed_data',
    from_json(col('data'), country_schema)
)

print("✓ JSON parseado com sucesso")


TRANSFORMAÇÃO: PARSE JSON
✓ JSON parseado com sucesso


## 5. Criando a Primeira Dimensão: dim_countries

### O que é uma dimensão?
Uma **dimensão** é uma tabela que descreve **"quem", "o quê", "onde", "quando"**. A `dim_countries` descreve **"onde"**: informações sobre cada país.

### Analogia:
Imagine uma planilha Excel onde cada linha é um país e cada coluna é uma característica:

| country_id | country_name | capital  | region        | landlocked |
|------------|--------------|----------|---------------|------------|
| BRA        | Brazil       | Brasília | South America | No         |
| USA        | United States| Washington DC | Americas  | No         |
| CHE        | Switzerland  | Bern     | Europe        | Yes        |

### O que este código faz?

**Passo 1: Extração (Flatten)**
- Pega os campos que nos interessam do JSON parseado
- "Achata" estruturas aninhadas (ex: `name.common` vira só `country_name_common`)
- Renomeia para nomes mais claros e padronizados
- Adiciona timestamp de quando foi criada

**Passo 2: Limpeza e Validação**
- **Remove países sem ID**: Se não tem identificador único, não podemos usá-lo
- **Remove países sem nome**: Não faz sentido ter um país sem nome
- **Remove duplicatas**: Se por algum motivo o mesmo país apareceu 2 vezes, mantemos apenas 1

**Passo 3: Visualização**
- Mostra 5 países como amostra para conferência visual
- Conta quantos países válidos temos (esperamos 195)

### Campos importantes:
- **country_id (cca3)**: Chave primária - código único de 3 letras (BRA, USA, CHN)
- **latitude/longitude**: Coordenadas geográficas do centro do país
- **landlocked**: True = sem costa marítima (ex: Bolívia, Suíça)
- **timezones**: Lista de fusos horários (Brasil tem 4)

### SCD Type 1:
Usamos "mode overwrite" porque estamos implementando **SCD Type 1**: mantemos apenas o estado atual, sem histórico. Se a população do Brasil mudar amanhã, simplesmente substituímos o valor antigo.


In [0]:
print("\n" + "="*60)
print("CRIANDO: dim_countries")
print("="*60)

# Flatten e selecionar campos
df_dim_countries = df_countries.select(
    col('parsed_data.cca3').alias('country_id'),
    col('parsed_data.cca2').alias('country_code_2'),
    col('parsed_data.name.common').alias('country_name_common'),
    col('parsed_data.name.official').alias('country_name_official'),
    col('parsed_data.capital')[0].alias('capital'),
    col('parsed_data.region').alias('region'),
    col('parsed_data.subregion').alias('subregion'),
    col('parsed_data.landlocked').alias('landlocked'),
    col('parsed_data.latlng')[0].alias('latitude'),
    col('parsed_data.latlng')[1].alias('longitude'),
    col('parsed_data.timezones').alias('timezones'),
    current_timestamp().alias('last_updated')
)

# Validações de qualidade
df_dim_countries_clean = df_dim_countries \
    .filter(col('country_id').isNotNull()) \
    .filter(col('country_name_common').isNotNull()) \
    .dropDuplicates(['country_id'])

# Visualizar amostra
print("\nAmostra da dimensão:")
df_dim_countries_clean.select(
    'country_id', 'country_name_common', 'capital', 'region', 'subregion'
).show(5, truncate=False)

dim_countries_count = df_dim_countries_clean.count()
print(f"\n✓ Registros na dimensão: {dim_countries_count}")


CRIANDO: dim_countries

Amostra da dimensão:
+----------+-------------------+------------+--------+-------------------------+
|country_id|country_name_common|capital     |region  |subregion                |
+----------+-------------------+------------+--------+-------------------------+
|ATG       |Antigua and Barbuda|Saint John's|Americas|Caribbean                |
|BTN       |Bhutan             |Thimphu     |Asia    |Southern Asia            |
|ITA       |Italy              |Rome        |Europe  |Southern Europe          |
|TUV       |Tuvalu             |Funafuti    |Oceania |Polynesia                |
|AUS       |Australia          |Canberra    |Oceania |Australia and New Zealand|
+----------+-------------------+------------+--------+-------------------------+
only showing top 5 rows

✓ Registros na dimensão: 195


### Salvando a Dimensão dim_countries

**O que este código faz?**
1. Pega a planilha limpa (`df_dim_countries_clean`)
2. Salva em formato Delta Lake
3. **Modo overwrite**: Substitui os dados antigos pelos novos (SCD Type 1)
4. Cria a tabela no catálogo como `workspace.silver.dim_countries`

**Por que overwrite e não append?**
- Na Bronze, usamos **append** porque queremos histórico (dados de ontem + dados de hoje)
- Na Silver (dimensões), usamos **overwrite** porque queremos apenas o estado atual
- Se rodarmos hoje e amanhã, os dados de amanhã substituem os de hoje

**Quando usar append vs overwrite:**
- **Append**: Acumular histórico (logs, transações, snapshots diários)
- **Overwrite**: Estado atual (dimensões, referências, cadastros)


In [0]:
# Salvar dimensão (modo overwrite para SCD Type 1)
(
    df_dim_countries_clean
    .write
    .format('delta')
    .mode('overwrite')
    .saveAsTable(SILVER_TABLES['dim_countries'])
)

print(f"\n✓ Dimensão salva: {SILVER_TABLES['dim_countries']}")
log_metrics('silver', SILVER_TABLES['dim_countries'], dim_countries_count)


✓ Dimensão salva: workspace.silver.dim_countries

MÉTRICAS - SILVER
Tabela: workspace.silver.dim_countries
Registros: 195
Timestamp: 2025-12-06 14:54:54.667857



## 6. Criando a Segunda Dimensão: dim_currencies

### O que é dim_currencies?
Uma tabela com todas as **moedas do mundo**: nome, código e símbolo.

### Exemplo visual:
| currency_code | currency_name      | currency_symbol |
|---------------|--------------------|-----------------|
| BRL           | Brazilian real     | R$              |
| USD           | United States dollar | $             |
| EUR           | Euro               | €               |
| JPY           | Japanese yen       | ¥               |

### O desafio: Dados aninhados
No JSON de cada país, as moedas vêm assim:
```json
"currencies": {
  "BRL": {"name": "Brazilian real", "symbol": "R$"},
  "USD": {"name": "United States dollar", "symbol": "$"}
}
```

É um **dicionário dentro de um dicionário**. Precisamos "explodir" isso!

### O que é "explode"?
**Explode** transforma um dicionário em múltiplas linhas:

**Antes:**
```
País: Brasil
Moedas: {BRL: {name: Real, symbol: R$}}
```

**Depois:**
```
Linha 1: BRL | Brazilian real | R$
```

### O que este código faz?

**Passo 1: Explodir o dicionário de moedas**
- Pega o campo `currencies` de cada país
- Transforma cada moeda em uma linha separada
- Extrai código (BRL), nome (Brazilian real) e símbolo (R$)

**Passo 2: Limpar e deduplicar**
- Remove moedas sem código (inválidas)
- Remove duplicatas: Euro aparece em 20 países, mas queremos apenas 1 linha

**Passo 3: Visualizar**
- Mostra 10 moedas como amostra
- Conta quantas moedas únicas existem (esperamos ~156)

### Por que deduplicar?
20 países da Europa usam o Euro. Se não deduplicas, teríamos 20 linhas idênticas com EUR. Queremos apenas 1!


In [0]:
print("\n" + "="*60)
print("CRIANDO: dim_currencies")
print("="*60)

# Explodir o map de currencies
df_currencies_exploded = df_countries.select(
    col('parsed_data.cca3').alias('country_id'),
    explode(col('parsed_data.currencies')).alias('currency_code', 'currency_data')
).select(
    'currency_code',
    col('currency_data.name').alias('currency_name'),
    col('currency_data.symbol').alias('currency_symbol')
)

# Remover duplicatas e nulos
df_dim_currencies = df_currencies_exploded \
    .filter(col('currency_code').isNotNull()) \
    .dropDuplicates(['currency_code'])

# Visualizar amostra
print("\nAmostra da dimensão:")
df_dim_currencies.show(10, truncate=False)

dim_currencies_count = df_dim_currencies.count()
print(f"\n✓ Registros na dimensão: {dim_currencies_count}")


CRIANDO: dim_currencies

Amostra da dimensão:
+-------------+------------------------+---------------+
|currency_code|currency_name           |currency_symbol|
+-------------+------------------------+---------------+
|EUR          |Euro                    |€              |
|LBP          |Lebanese pound          |ل.ل            |
|KZT          |Kazakhstani tenge       |₸              |
|USD          |United States dollar    |$              |
|XCD          |Eastern Caribbean dollar|$              |
|AOA          |Angolan kwanza          |Kz             |
|RON          |Romanian leu            |lei            |
|RSD          |Serbian dinar           |дин.           |
|DZD          |Algerian dinar          |د.ج            |
|KWD          |Kuwaiti dinar           |د.ك            |
+-------------+------------------------+---------------+
only showing top 10 rows

✓ Registros na dimensão: 146


### Salvando a Dimensão dim_currencies

**O que este código faz?**
Mesma operação que fizemos com `dim_countries`:
1. Salva em Delta Lake
2. Modo overwrite (estado atual, sem histórico)
3. Registra no catálogo como `workspace.silver.dim_currencies`
4. Exibe métricas de sucesso

**Resultado esperado:**
Uma tabela com aproximadamente 156 moedas únicas do mundo, pronta para ser usada em análises sobre zonas monetárias e câmbio.


In [0]:
# Salvar dimensão
(
    df_dim_currencies
    .write
    .format('delta')
    .mode('overwrite')
    .saveAsTable(SILVER_TABLES['dim_currencies'])
)

print(f"\n✓ Dimensão salva: {SILVER_TABLES['dim_currencies']}")
log_metrics('silver', SILVER_TABLES['dim_currencies'], dim_currencies_count)


✓ Dimensão salva: workspace.silver.dim_currencies

MÉTRICAS - SILVER
Tabela: workspace.silver.dim_currencies
Registros: 146
Timestamp: 2025-12-06 14:55:01.026795



## 7. Criando a Terceira Dimensão: dim_languages

### O que é dim_languages?
Uma tabela com todos os **idiomas do mundo**: código e nome.

### Exemplo visual:
| language_code | language_name |
|---------------|---------------|
| por           | Portuguese    |
| eng           | English       |
| spa           | Spanish       |
| fra           | French        |
| zho           | Chinese       |

### Estrutura no JSON:
Assim como moedas, idiomas também vêm aninhados:
```json
"languages": {
  "por": "Portuguese",
  "eng": "English"
}
```

### O que este código faz?

**Passo 1: Explodir o dicionário de idiomas**
- Transforma o dicionário em linhas separadas
- Extrai código (por) e nome (Portuguese)

**Passo 2: Limpar e deduplicar**
- Remove idiomas sem código
- Remove duplicatas: Inglês aparece em 67 países, queremos apenas 1 linha

**Passo 3: Visualizar**
- Mostra 10 idiomas como amostra
- Conta quantos idiomas únicos existem (esperamos ~490)

### Curiosidade:
Muitos países são multilíngues:
- **Canadá**: Inglês e Francês
- **Suíça**: Alemão, Francês, Italiano e Romanche
- **Índia**: 22 idiomas oficiais!

Explode garante que cada idioma vire uma linha individual.


In [0]:
print("\n" + "="*60)
print("CRIANDO: dim_languages")
print("="*60)

# Explodir o map de languages
df_languages_exploded = df_countries.select(
    explode(col('parsed_data.languages')).alias('language_code', 'language_name')
)

# Remover duplicatas e nulos
df_dim_languages = df_languages_exploded \
    .filter(col('language_code').isNotNull()) \
    .dropDuplicates(['language_code'])

# Visualizar amostra
print("\nAmostra da dimensão:")
df_dim_languages.show(10, truncate=False)

dim_languages_count = df_dim_languages.count()
print(f"\n✓ Registros na dimensão: {dim_languages_count}")


CRIANDO: dim_languages

Amostra da dimensão:
+-------------+-------------+
|language_code|language_name|
+-------------+-------------+
|lit          |Lithuanian   |
|ara          |Arabic       |
|fra          |French       |
|kaz          |Kazakh       |
|rus          |Russian      |
|por          |Portuguese   |
|tet          |Tetum        |
|eng          |English      |
|ron          |Romanian     |
|srp          |Serbian      |
+-------------+-------------+
only showing top 10 rows

✓ Registros na dimensão: 140


### Salvando a Dimensão dim_languages

**O que este código faz?**
Salva a dimensão de idiomas seguindo o mesmo padrão:
1. Delta Lake format
2. Modo overwrite
3. Registra como `workspace.silver.dim_languages`
4. Log de métricas

**Resultado esperado:**
Aproximadamente 490 idiomas únicos catalogados, prontos para análises de diversidade linguística global.


In [0]:
# Salvar dimensão
(
    df_dim_languages
    .write
    .format('delta')
    .mode('overwrite')
    .saveAsTable(SILVER_TABLES['dim_languages'])
)

print(f"\n✓ Dimensão salva: {SILVER_TABLES['dim_languages']}")
log_metrics('silver', SILVER_TABLES['dim_languages'], dim_languages_count)


✓ Dimensão salva: workspace.silver.dim_languages

MÉTRICAS - SILVER
Tabela: workspace.silver.dim_languages
Registros: 140
Timestamp: 2025-12-06 14:55:06.915690



## 8. Criando a Tabela Fato: fact_country_metrics

### O que é uma tabela fato?
Se as dimensões descrevem **"quem/o quê/onde"**, a tabela fato guarda as **métricas e medidas numéricas**. É o "coração" do Star Schema.

### Analogia:
Imagine uma planilha de vendas:
- **Dimensões**: Cliente (quem), Produto (o quê), Loja (onde), Data (quando)
- **Fato**: Quantidade vendida, Valor total (métricas numéricas)

No nosso caso:
- **Dimensão**: País (dim_countries)
- **Fato**: População, Área, Densidade populacional

### Exemplo visual:
| country_id | population  | area_km2   | population_density | snapshot_date |
|------------|-------------|------------|-------------------|---------------|
| BRA        | 213,993,437 | 8,515,767  | 25.13             | 2025-12-05    |
| USA        | 331,900,000 | 9,833,520  | 33.74             | 2025-12-05    |
| CHN        | 1,412,000,000| 9,596,961 | 147.17            | 2025-12-05    |

### O que este código faz?

**Passo 1: Extrair métricas básicas**
- `country_id`: Chave estrangeira ligando ao dim_countries
- `population`: Número de habitantes
- `area_km2`: Área territorial em quilômetros quadrados
- `snapshot_date`: Data de hoje (quando capturamos esses números)

**Passo 2: Calcular densidade populacional**
- **Fórmula**: `população ÷ área`
- **Exemplo**: Brasil tem 213M habitantes em 8.5M km² = 25 pessoas/km²
- **Tratamento especial**: Se área = 0, densidade = 0 (evita divisão por zero)

**Passo 3: Validações**
- População deve ser ≥ 0 (não existe população negativa)
- Área deve ser ≥ 0 (não existe área negativa)
- Remove registros inválidos

**Passo 4: Visualizar**
- Ordena por população (maior → menor)
- Mostra os 10 países mais populosos
- Conta total de registros válidos

### Por que calcular densidade?
Densidade populacional é uma métrica derivada extremamente útil:
- **Alta densidade** (ex: Bangladesh = 1.265/km²): Urbanização intensa, desafios de infraestrutura
- **Baixa densidade** (ex: Mongólia = 2/km²): Vastidões, desafios logísticos

### Snapshot date:
Guardamos a data para saber "quando" esses números eram válidos. Se rodarmos o pipeline daqui a 6 meses, teremos um novo snapshot com números atualizados.


In [0]:
print("\n" + "="*60)
print("CRIANDO: fact_country_metrics")
print("="*60)

# Criar tabela fato com métricas
df_fact_metrics = df_countries.select(
    col('parsed_data.cca3').alias('country_id'),
    col('parsed_data.population').alias('population'),
    col('parsed_data.area').alias('area_km2'),
    lit(date.today()).alias('snapshot_date')
).filter(
    col('country_id').isNotNull()
)

# Calcular densidade populacional
df_fact_metrics = df_fact_metrics.withColumn(
    'population_density',
    when(col('area_km2') > 0, col('population') / col('area_km2')).otherwise(0)
)

# Validações (população e área devem ser >= 0)
df_fact_metrics_clean = df_fact_metrics \
    .filter(col('population') >= 0) \
    .filter(col('area_km2') >= 0)

# Visualizar amostra
print("\nAmostra da tabela fato:")
df_fact_metrics_clean.orderBy(col('population').desc()).show(10, truncate=False)

fact_metrics_count = df_fact_metrics_clean.count()
print(f"\n✓ Registros na tabela fato: {fact_metrics_count}")


CRIANDO: fact_country_metrics

Amostra da tabela fato:
+----------+----------+-----------+-------------+------------------+
|country_id|population|area_km2   |snapshot_date|population_density|
+----------+----------+-----------+-------------+------------------+
|IND       |1417492000|3287263.0  |2025-12-06   |431.2073600439028 |
|CHN       |1408280000|9706961.0  |2025-12-06   |145.07939199508476|
|USA       |340110988 |9525067.0  |2025-12-06   |35.70693917428612 |
|IDN       |284438782 |1904569.0  |2025-12-06   |149.34548551404544|
|PAK       |241499431 |796095.0   |2025-12-06   |303.3550405416439 |
|NGA       |223800000 |923768.0   |2025-12-06   |242.26862155865976|
|BRA       |213421037 |8515767.0  |2025-12-06   |25.061869001347734|
|BGD       |169828911 |147570.0   |2025-12-06   |1150.8362878633868|
|RUS       |146028325 |1.7098246E7|2025-12-06   |8.540544158740024 |
|MEX       |130575786 |1964375.0  |2025-12-06   |66.47192414890232 |
+----------+----------+-----------+------------

### Salvando a Tabela Fato

**O que este código faz?**
Salva a tabela fato com métricas calculadas:
1. Delta Lake format
2. Modo overwrite (SCD Type 1 - snapshot atual)
3. Registra como `workspace.silver.fact_country_metrics`
4. Log de métricas

**Resultado esperado:**
195 registros (um para cada país) com população, área e densidade calculada, prontos para análises quantitativas.

**Futuro: SCD Type 2**
Se quiséssemos rastrear mudanças ao longo do tempo (população crescendo, densidade mudando), usaríamos:
- Modo append (acumular snapshots)
- Versioning (validFrom, validTo)
- Permite análises como "como a densidade da China mudou nos últimos 10 anos?"


In [0]:
# Salvar tabela fato
(
    df_fact_metrics_clean
    .write
    .format('delta')
    .mode('overwrite')
    .saveAsTable(SILVER_TABLES['fact_country_metrics'])
)

print(f"\n✓ Tabela fato salva: {SILVER_TABLES['fact_country_metrics']}")
log_metrics('silver', SILVER_TABLES['fact_country_metrics'], fact_metrics_count)


✓ Tabela fato salva: workspace.silver.fact_country_metrics

MÉTRICAS - SILVER
Tabela: workspace.silver.fact_country_metrics
Registros: 195
Timestamp: 2025-12-06 14:55:12.553274



## 9. Validações de Qualidade de Dados

### Por que validar?
Agora que transformamos e salvamos os dados, precisamos **garantir que tudo está correto**. É como fazer controle de qualidade em uma fábrica: verificamos se não há peças defeituosas.

### Validações que faremos:

#### 1️⃣ Integridade Referencial
**O que é?** Verificar se todos os `country_id` da tabela fato existem na dimensão países.

**Por que importa?** 
- Imagine que o fato diz: "País XYZ tem 10M de habitantes"
- Mas quando vamos procurar XYZ no dim_countries, não existe!
- Isso quebra as análises (joins falham)

**O que esperamos:** 0 registros órfãos (todos os IDs do fato devem existir na dimensão)

**Analogia:** É como uma lista de pedidos referenciando produtos. Se um pedido menciona "Produto ABC" mas esse produto não existe no catálogo, temos um problema.

---

#### 2️⃣ Validação de Ranges (Faixas Válidas)
**O que é?** Verificar se valores numéricos estão em faixas lógicas.

**Regras de negócio:**
- **População ≥ 0**: Não existe população negativa
- **Área ≥ 0**: Não existe área negativa

**O que esperamos:** 0 valores inválidos (já limpamos na etapa anterior, mas é bom confirmar)

**Outros checks possíveis (não implementados aqui):**
- Latitude entre -90 e 90
- Longitude entre -180 e 180
- Densidade < 50.000 (maior densidade real é ~22.000 em Macau)

---

#### 3️⃣ Estatísticas Gerais
**O que é?** Calcular médias para ter uma visão geral dos dados.

**Métricas calculadas:**
- **População média**: ~40 milhões (alguns gigantes como China e Índia puxam a média para cima)
- **Área média**: ~700.000 km² (Rússia e Canadá aumentam muito essa média)
- **Densidade média**: ~50-60 habitantes/km²

**Por que calcular médias?**
- Baseline para comparações: "País X tem densidade acima ou abaixo da média?"
- Detectar outliers: Se a média mudar drasticamente, pode indicar erro nos dados
- Contexto para análises: Entender a "normalidade" dos dados

---

### O que este código faz?
1. **Recarrega as tabelas** Silver do banco de dados
2. **Faz um left_anti join**: Encontra IDs no fato que NÃO existem na dimensão
3. **Conta registros órfãos**: Esperamos 0
4. **Filtra valores inválidos**: População < 0 ou Área < 0
5. **Calcula estatísticas**: Médias de população, área e densidade
6. **Mostra tudo na tela**: Para inspeção visual


In [0]:
print("\n" + "="*60)
print("VALIDAÇÕES DE QUALIDADE")
print("="*60)

# Check 1: Integridade referencial (todos os country_id do fato existem na dimensão)
df_countries_dim = spark.table(SILVER_TABLES['dim_countries'])
df_metrics_fact = spark.table(SILVER_TABLES['fact_country_metrics'])

orphan_records = df_metrics_fact.join(
    df_countries_dim,
    df_metrics_fact.country_id == df_countries_dim.country_id,
    'left_anti'
).count()

print(f"\n✓ Check Integridade Referencial: {orphan_records} registros órfãos")

# Check 2: Validar ranges
invalid_population = df_metrics_fact.filter(col('population') < 0).count()
invalid_area = df_metrics_fact.filter(col('area_km2') < 0).count()

print(f"✓ Check População: {invalid_population} valores inválidos")
print(f"✓ Check Área: {invalid_area} valores inválidos")

# Check 3: Estatísticas gerais
print("\nEstatísticas gerais:")
df_metrics_fact.select(
    round(avg('population'), 0).alias('avg_population'),
    round(avg('area_km2'), 2).alias('avg_area_km2'),
    round(avg('population_density'), 2).alias('avg_density')
).show()


VALIDAÇÕES DE QUALIDADE

✓ Check Integridade Referencial: 0 registros órfãos
✓ Check População: 0 valores inválidos
✓ Check Área: 0 valores inválidos

Estatísticas gerais:
+--------------+------------+-----------+
|avg_population|avg_area_km2|avg_density|
+--------------+------------+-----------+
|    4.089452E7|    684964.2|     311.54|
+--------------+------------+-----------+



## 10. Otimização das Tabelas Silver

### O que é otimização?
Mesma operação que fizemos na Bronze: **compactar arquivos pequenos em arquivos maiores** para melhorar performance de leitura.

### Por que otimizar as 4 tabelas?
- `dim_countries` (195 registros): Pode ter sido salva em vários arquivos pequenos
- `dim_currencies` (156 registros): Idem
- `dim_languages` (490 registros): Idem
- `fact_country_metrics` (195 registros): Idem

OPTIMIZE junta tudo em poucos arquivos otimizados.

### Quando o ganho é significativo?
- **Agora**: Ganho pequeno (poucas linhas)
- **Depois de 1 ano rodando diariamente**: Ganho enorme
- **Com milhões de registros**: Essencial para performance

### O que este código faz?
1. Roda comando OPTIMIZE em cada uma das 4 tabelas Silver
2. Não usa Z-ORDER (seria aplicado em colunas específicas, não necessário aqui)
3. Mostra confirmação de sucesso para cada tabela


In [0]:
print("\nOtimizando tabelas Silver...\n")

# Otimizar sem Z-ORDER para evitar problemas
optimize_table(spark, SILVER_TABLES['dim_countries'])
optimize_table(spark, SILVER_TABLES['dim_currencies'])
optimize_table(spark, SILVER_TABLES['dim_languages'])
optimize_table(spark, SILVER_TABLES['fact_country_metrics'])

print("\n✓ Otimização concluída")


Otimizando tabelas Silver...

Otimizando tabela workspace.silver.dim_countries...
Tabela workspace.silver.dim_countries otimizada com sucesso
Otimizando tabela workspace.silver.dim_currencies...
Tabela workspace.silver.dim_currencies otimizada com sucesso
Otimizando tabela workspace.silver.dim_languages...
Tabela workspace.silver.dim_languages otimizada com sucesso
Otimizando tabela workspace.silver.fact_country_metrics...
Tabela workspace.silver.fact_country_metrics otimizada com sucesso

✓ Otimização concluída


## 11. Resumo Final da Execução Silver

### O que este código faz?
Gera um **relatório completo** de tudo que foi realizado no pipeline Silver:

1. **Tabelas criadas**: Lista todas as 4 tabelas com quantidade de registros
2. **Validações realizadas**: Confirma que todas passaram
3. **Tempo de execução**: Quanto tempo levou todo o processamento
4. **Status final**: SUCESSO ou erro
5. **Timestamp**: Quando terminou

### Por que este resumo é importante?
- **Documentação**: Registro de que o pipeline rodou com sucesso
- **Monitoramento**: Se o tempo aumentar muito, investiga-se o motivo
- **Auditoria**: Prova de que as transformações foram aplicadas
- **Troubleshooting**: Se algo falhar, sabemos exatamente o que foi completado

### Output esperado:
```
PIPELINE SILVER - RESUMO FINAL
Tabelas criadas:
  1. workspace.silver.dim_countries (195 registros)
  2. workspace.silver.dim_currencies (156 registros)
  3. workspace.silver.dim_languages (490 registros)
  4. workspace.silver.fact_country_metrics (195 registros)
Validações:
  - Integridade referencial: ✓
  - Validação de ranges: ✓
  - Deduplicação: ✓
Tempo total: 45.32 segundos
Status: ✓ SUCESSO
```


In [0]:
execution_time = time.time() - start_time

print("\n" + "="*60)
print("PIPELINE SILVER - RESUMO FINAL")
print("="*60)
print(f"\nTabelas criadas:")
print(f"  1. {SILVER_TABLES['dim_countries']} ({dim_countries_count:,} registros)")
print(f"  2. {SILVER_TABLES['dim_currencies']} ({dim_currencies_count:,} registros)")
print(f"  3. {SILVER_TABLES['dim_languages']} ({dim_languages_count:,} registros)")
print(f"  4. {SILVER_TABLES['fact_country_metrics']} ({fact_metrics_count:,} registros)")
print(f"\nValidações:")
print(f"  - Integridade referencial: ✓")
print(f"  - Validação de ranges: ✓")
print(f"  - Deduplicação: ✓")
print(f"\nTempo total de execução: {execution_time:.2f} segundos")
print(f"Status: ✓ SUCESSO")
print(f"Timestamp final: {datetime.now()}")
print("="*60)


PIPELINE SILVER - RESUMO FINAL

Tabelas criadas:
  1. workspace.silver.dim_countries (195 registros)
  2. workspace.silver.dim_currencies (146 registros)
  3. workspace.silver.dim_languages (140 registros)
  4. workspace.silver.fact_country_metrics (195 registros)

Validações:
  - Integridade referencial: ✓
  - Validação de ranges: ✓
  - Deduplicação: ✓

Tempo total de execução: 37.30 segundos
Status: ✓ SUCESSO
Timestamp final: 2025-12-06 14:55:21.582096
